# Objective: To forecast the Grab riding demand over the 1329 geohash coordinates in each 15-min time frame, experiments was done based on the models RMSE results which perform the best for the forecasting in 672 test samples (1week in future) 
## Throughout my experiment, the best model is VAR model with 672  lags and Weekday Exogenous variables. The algorithm still could running faster with such large lags with thousands variable are due to the application of PCA decomposion which reduced the data dimension while still preserving a high variation of the data 
## As conclusion, this could be implies that sometimes the traditional statistical forecasting approach can still work better than the complicated deep learning method. Further more the traditional statistical forecasting could be more interpretation due to it simplicity and model coefficients. 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
from sklearn.decomposition import PCA
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error,mean_absolute_error
from statsmodels.tools.eval_measures import rmse
pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 50)

# Data Preprocessing
## Regroup the data to feed in the modelling algorithm, find out the missing timestamp which occurs when all of the location had 0 demand in certain time (probably due to system down) and imputing these missing value to ensure the time series are in order

In [2]:
Grab = pd.read_csv('GrabAssignment.csv')

In [3]:
Grab.head()

,geohash6,day,timestamp,demand
0,qp03wc,18,20:0,0.020072
1,qp03pn,10,14:30,0.024721
2,qp09sw,9,6:15,0.102821
3,qp0991,32,5:0,0.088755
4,qp090q,15,4:0,0.074468


In [4]:
Grab.shape

(4206321, 4)

In [5]:
Grab.columns

Index(['geohash6', 'day', 'timestamp', 'demand'], dtype='object')

In [6]:
Grab.dtypes

geohash6      object
day            int64
timestamp     object
demand       float64
dtype: object

In [7]:
# Each day should have 96 timestamp (15 min), here to check out which day didnt have complete timestamp cycle
a_list = []
for c in range(1,61):
    lens = len(Grab[Grab["day"] == c].timestamp.value_counts())
    if lens != 96:
        a_list.append(c)
print(a_list)

[18]


In [8]:
#Regroup data to make the day and timestamp in order and unstack geohash
GrabFull = Grab.groupby(['geohash6', 'day', 'timestamp'])['demand'].mean().unstack('geohash6')

In [9]:
GrabFull.columns

Index(['qp02yc', 'qp02yf', 'qp02yu', 'qp02yv', 'qp02yy', 'qp02yz', 'qp02z1',
       'qp02z3', 'qp02z4', 'qp02z5',
       ...
       'qp0djv', 'qp0djw', 'qp0djy', 'qp0dn0', 'qp0dn1', 'qp0dn4', 'qp0dn5',
       'qp0dnh', 'qp0dnj', 'qp0dnn'],
      dtype='object', name='geohash6', length=1329)

In [10]:
# Creating the DataFrame to fillup the NaN value in Raw Data to complete timestamp cycle for timeseries forecasting
fillup_ts = pd.DataFrame(GrabFull.columns)
fillup_ts["day"] = 18
fillup_ts["demand"] = np.NaN

In [11]:
fillup_ts

,geohash6,day,demand
0,qp02yc,18,NaN
1,qp02yf,18,NaN
2,qp02yu,18,NaN
3,qp02yv,18,NaN
4,qp02yy,18,NaN
5,qp02yz,18,NaN
6,qp02z1,18,NaN
7,qp02z3,18,NaN
8,qp02z4,18,NaN
9,qp02z5,18,NaN


In [12]:
fillup_ts_str = pd.Series([str("9:45"),str("10:0"),str("10:15"),str("12:45"),str("11:30"),str("11:45"),str("12:0"),str("12:15"),str("12:30")])

In [13]:
fillup_ts_full = pd.DataFrame()
for ts in fillup_ts_str:
    fillup_ts["timestamp"] = ts
    fillup_ts_full = pd.concat([fillup_ts_full,fillup_ts], ignore_index=True)
    

In [14]:
fillup_ts_full

,geohash6,day,demand,timestamp
0,qp02yc,18,NaN,9:45
1,qp02yf,18,NaN,9:45
2,qp02yu,18,NaN,9:45
3,qp02yv,18,NaN,9:45
4,qp02yy,18,NaN,9:45
5,qp02yz,18,NaN,9:45
6,qp02z1,18,NaN,9:45
7,qp02z3,18,NaN,9:45
8,qp02z4,18,NaN,9:45
9,qp02z5,18,NaN,9:45


In [15]:
Grab_done = pd.concat([Grab,fillup_ts_full], ignore_index=True, sort=False)

In [16]:
Grab_done

,geohash6,day,timestamp,demand
0,qp03wc,18,20:0,0.020072
1,qp03pn,10,14:30,0.024721
2,qp09sw,9,6:15,0.102821
3,qp0991,32,5:0,0.088755
4,qp090q,15,4:0,0.074468
5,qp03tu,1,12:15,0.023843
6,qp096d,25,3:30,0.007460
7,qp03nr,51,20:45,0.000293
8,qp093r,48,6:15,0.054170
9,qp03r2,4,22:15,0.123463


In [17]:
Grab_done['timestamp'] = pd.to_datetime(Grab_done['timestamp'], format='%H:%M').dt.time

In [18]:
grab_train = Grab_done.groupby(['geohash6', 'day', 'timestamp'])['demand'].mean().unstack('geohash6')

In [19]:
grab_train.shape

(5856, 1329)

In [20]:
grab_train.isnull().sum().sum()

3576303

In [21]:
#Merge the consecutive day and timestamp into one time series with a assigned datetime and 15min frequency
grab_train = grab_train.set_index(pd.date_range(datetime(2019, 1, 1, hour=0, minute=0), periods=5856, freq='15min'))

In [22]:
grab_train = grab_train.fillna(0)

In [23]:
grab_train.head(10)

geohash6,qp02yc,qp02yf,qp02yu,qp02yv,qp02yy,qp02yz,qp02z1,qp02z3,qp02z4,qp02z5,qp02z6,qp02z7,qp02z9,qp02zc,qp02zd,qp02ze,qp02zf,qp02zg,qp02zh,qp02zj,qp02zk,qp02zm,qp02zn,qp02zp,qp02zq,...,qp0djb,qp0djc,qp0djd,qp0dje,qp0djf,qp0djg,qp0djh,qp0djj,qp0djk,qp0djm,qp0djn,qp0djq,qp0djs,qp0djt,qp0dju,qp0djv,qp0djw,qp0djy,qp0dn0,qp0dn1,qp0dn4,qp0dn5,qp0dnh,qp0dnj,qp0dnn
2019-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.022396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.009482,0.0,0.000000,0.000000,0.003641,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0006,0.0,0.000000,0.000000,0.038979,0.000000,0.010554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.012865,0.000000,0.000000,0.0,0.001304,0.000000,0.002454,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.004056,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.022022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.004910,0.000000,0.000000,0.0,0.020167,0.000000,0.000000,0.000000,0.0,0.000503,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.009381,0.0
2019-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.070525,0.001295,0.000000,0.000000,0.008541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.001248,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.083947,0.000000,0.045482,0.047912,0.006503,0.000000,0.000000,0.000000,0.049005,0.047575,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.009687,0.000000,0.051739,0.0,0.010428,0.000000,0.000000,0.0,0.0,0.0,0.008253,0.0,0.0,0.0,0.0,0.0,0.002701,0.0
2019-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.000000,0.045083,0.000000,0.000000,0.031414,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060752,0.000000,0.000000,...,0.0,0.0,0.011394,0.000000,0.000000,0.0,0.009599,0.017358,0.000000,0.000000,0.0,0.020719,0.000000,0.000000,0.0,0.0,0.0,0.006971,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.099317,0.0,0.0,0.0000,0.0,0.000000,0.000000,0.000000,0.048360,0.000954,0.000000,0.000000,0.000000,0.000000,0.000000,0.104351,0.075672,0.000630,0.066387,...,0.0,0.0,0.000000,0.000471,0.012844,0.0,0.027026,0.029316,0.007121,0.005744,0.0,0.000000,0.000993,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.019563,0.0
2019-01-01 01:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.011855,0.003631,0.000000,0.000000,0.039394,0.000000,0.000000,0.000000,0.000000,0.019048,0.036850,0.079985,0.055821,0.071413,...,0.0,0.0,0.000000,0.000000,0.003883,0.0,0.004794,0.000000,0.019596,0.031227,0.0,0.023070,0.000721,0.000000,0.0,0.0,0.0,0.001858,0.0,0.0,0.0,0.0,0.0,0.007193,0.0
2019-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.029813,0.0,0.0,0.0000,0.0,0.000000,0.045931,0.000000,0.000000,0.000000,0.000000,0.023930,0.000000,0.082367,0.000000,0.019131,0.022062,0.036493,0.012899,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.016685,0.024294,0.034116,0.018435,0.0,0.011025,0.005066,0.011381,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2019-01-01 02:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.099991,0.0,0.0,0.0000,0.0,0.019614,0.019863,0.000000,0.016487,0.000000,0.000000,0.044045,0.014075,0.237685,0.000000,0.042676,0.124104,0.038605,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000553,0.006702,0.032528,0.007979,0.0,0.000000,0.019812,0.002717,0.0,0.0,0.0,0.008936,0.0,0.0,0.0,0.0,0.0,0.007224,0.0


# Apply PCA decomposition to reduce the data dimension while maintaining the most data variation

In [24]:
from sklearn.decomposition import PCA

In [25]:
pca = PCA(0.95)

In [26]:
train_2D = pca.fit_transform(grab_train[:-672])

In [27]:
pca.explained_variance_ratio_.sum()

0.9501405036970398

In [28]:
pca.explained_variance_ratio_

array([6.51803521e-01, 6.13084516e-02, 4.39834237e-02, 2.70607501e-02,
       2.43508716e-02, 1.75100875e-02, 1.29762158e-02, 8.40748785e-03,
       6.61011213e-03, 6.24463943e-03, 4.81676383e-03, 4.72410334e-03,
       3.87567708e-03, 3.64940584e-03, 3.12047035e-03, 2.69894027e-03,
       2.48680843e-03, 2.22087156e-03, 2.08501822e-03, 2.01848902e-03,
       1.88598121e-03, 1.63494691e-03, 1.52169704e-03, 1.50807216e-03,
       1.41445432e-03, 1.32655326e-03, 1.28530012e-03, 1.20351730e-03,
       1.16560559e-03, 1.12709808e-03, 1.10392382e-03, 1.06164220e-03,
       9.99131901e-04, 9.80129237e-04, 9.58076150e-04, 9.36822241e-04,
       8.99536398e-04, 8.77785092e-04, 8.50523352e-04, 8.15951215e-04,
       7.82523108e-04, 7.78746326e-04, 7.59420882e-04, 7.41447503e-04,
       7.20550428e-04, 7.11871780e-04, 6.90015780e-04, 6.76431463e-04,
       6.67984519e-04, 6.57261678e-04, 6.38505613e-04, 6.21709019e-04,
       6.09059280e-04, 5.98429760e-04, 5.90245775e-04, 5.80509339e-04,
      

In [29]:
pca.explained_variance_ratio_.shape

(117,)

In [30]:
train_2D = pd.DataFrame(train_2D)

In [31]:
train_2D = train_2D.set_index(pd.date_range(datetime(2019, 1, 1, hour=0, minute=0), periods=5184, freq='15min'))

In [32]:
train_2D.shape

(5184, 117)

# VAR modelling with 672 lags and Exogenous variable: Weekday

In [33]:
#Let the test dataset be one week dataset as it is the cycle of weekday & weekend
nobs=672    
train_11 = train_2D
test_11 = grab_train[-nobs:]

In [34]:
train_11_Exo_feature = train_11.copy()
test_11_Exo_feature = test_11.copy()

In [35]:
train_11_Exo_feature['Weekday'] = train_11_Exo_feature.index.weekday_name
test_11_Exo_feature['Weekday'] = test_11_Exo_feature.index.weekday_name

In [36]:
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

In [37]:
train_11_Exo_feature = create_dummies(train_11_Exo_feature,'Weekday')
test_11_Exo_feature = create_dummies(test_11_Exo_feature,'Weekday')

In [38]:
test_11_Exo_feature.columns

Index(['qp02yc', 'qp02yf', 'qp02yu', 'qp02yv', 'qp02yy', 'qp02yz', 'qp02z1',
       'qp02z3', 'qp02z4', 'qp02z5',
       ...
       'qp0dnj', 'qp0dnn', 'Weekday', 'Weekday_Friday', 'Weekday_Monday',
       'Weekday_Saturday', 'Weekday_Sunday', 'Weekday_Thursday',
       'Weekday_Tuesday', 'Weekday_Wednesday'],
      dtype='object', length=1337)

In [39]:
train_11_Exo_feature = train_11_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']
test_11_Exo_feature = test_11_Exo_feature.loc[:,'Weekday_Friday':'Weekday_Wednesday']

In [40]:
train_11_Exo_feature.sample(5)

,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
2019-01-23 21:30:00,0,0,0,0,0,0,1
2019-01-01 09:45:00,0,0,0,0,0,1,0
2019-01-20 14:30:00,0,0,0,1,0,0,0
2019-02-18 15:30:00,0,1,0,0,0,0,0
2019-02-06 08:45:00,0,0,0,0,0,0,1


In [41]:
model_VAR_11 = VAR(train_11, exog=train_11_Exo_feature)

In [42]:
results_model_VAR_11 = model_VAR_11.fit(maxlags=672, trend="ctt") 

In [43]:
results_model_VAR_11.aic

-2672.2772384458553

In [44]:
# Use previous Week data (672 in 15-min time stamp) to forecast the next value
lagged_values_VAR_11 = train_11.values[-672:]

In [45]:
z_F_11 = results_model_VAR_11.forecast(y=lagged_values_VAR_11, steps=672, exog_future=test_11_Exo_feature) 
# transform the forecasting values back to original dimension to compare the result
df_forecast_VAR_11 = pca.inverse_transform(z_F_11)

In [46]:
#Set the time frame to be same as test dataset
idxH_F_Exo = pd.date_range(datetime(2019, 2, 24, hour=0, minute=0), periods=672, freq='15min')

#Modify variable z to be a data frame and have the same time frame and columns with the test dataset rather than a array
df_forecast_VAR_11 = pd.DataFrame(df_forecast_VAR_11, index=idxH_F_Exo, columns=test_11.columns)

In [47]:
#Refine the forecast value by setting the demand value between 0 and 1
df_forecast_VAR_11[df_forecast_VAR_11 < 0] = 0
df_forecast_VAR_11[df_forecast_VAR_11 > 1] = 1

In [48]:
#The RMSE for the forecasted values and test dataset
RMSE = np.sqrt(mean_squared_error(test_11,df_forecast_VAR_11))
MAE = mean_absolute_error(test_11,df_forecast_VAR_11)*100
Test_dataset_mean = test_11.mean().sum()
RMSE_divide_Test_mean = (RMSE / Test_dataset_mean) * 100
relative_error_in_100_percentage = 100 - RMSE_divide_Test_mean
print(RMSE)
print(MAE)
print(Test_dataset_mean)
print(RMSE_divide_Test_mean)
print(relative_error_in_100_percentage)

0.041840895707627154
2.1388077664917358
81.90959266258383
0.05108180171275593
99.94891819828725
